<a href="https://colab.research.google.com/github/mohamedhossam822/Chest_X-ray_Diagnosis/blob/main/Pattern_recognition_COVID_19_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Downloading dataset**

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hosamtarek","key":"e876952ca558c43f1aa900273ae407e6"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database
from zipfile import ZipFile
file_name = "covid19-radiography-database.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')
  

 99% 1.13G/1.14G [00:09<00:00, 120MB/s] 
100% 1.14G/1.14G [00:09<00:00, 127MB/s]


# **Import libraries**

In [ ]:
import cv2 
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import torchvision.transforms as transforms
import torchvision
from sklearn.metrics import f1_score,accuracy_score

# **Reading images**

## Covid Data

In [ ]:
img_matrix=[]
# 60-20-20 ////////// 70-15-15
covid_folder_path="/content/COVID-19 Radiography Database/COVID"
Normal_folder_path="/content/COVID-19 Radiography Database/NORMAL"
Viral_pneumonia_path="/content/COVID-19 Radiography Database/Viral Pneumonia"

covid_train=[]
covid_validate=[]
covid_test=[]

i=1
for file in os.listdir(covid_folder_path):
     
    if file.endswith(".png"):
      img=cv2.imread(os.path.join("/content/COVID-19 Radiography Database/COVID", file))
      dim = (256, 256)
      img=cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
      if i<=(60/100)*1200:
        covid_train.append(img)
        #covid_train.append(cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)) # rotate img then append it to train set
        #covid_train.append(cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))       
        i+=1
      elif i<=(80/100)*1200:
        covid_validate.append(img)
        i+=1
      else:
        covid_test.append(img)
        i+=1

covid_train=np.array(covid_train)
covid_validate=np.array(covid_validate)
covid_test=np.array(covid_test)
print("Done")

Done


## Normal Data

In [ ]:
Normal_train=[]
Normal_validate=[]
Normal_test=[]
i=1
for file in os.listdir("/content/COVID-19 Radiography Database/NORMAL"):
    if file.endswith(".png"):
      img=cv2.imread(os.path.join("/content/COVID-19 Radiography Database/NORMAL", file))
      dim = (256, 256)
      img=cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
      if i<(60/100)*1341:
        Normal_train.append(img)
        #Normal_train.append(cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE))
        #Normal_train.append(cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
        i+=1
      elif i<(80/100)*1341:
        Normal_validate.append(img)
        i+=1
      else:
        Normal_test.append(img)
        i+=1
Normal_train=np.array(Normal_train)
Normal_validate=np.array(Normal_validate)
Normal_test=np.array(Normal_test)
print("Done")

Done


## Viral Data

In [ ]:
Viral_train=[]
Viral_validate=[]
Viral_test=[]
i=1
for file in os.listdir("/content/COVID-19 Radiography Database/Viral Pneumonia"):
    if file.endswith(".png"):
      img=cv2.imread(os.path.join("/content/COVID-19 Radiography Database/Viral Pneumonia", file))
      dim = (256, 256)
      img=cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
      if i<(60/100)*1346:
        Viral_train.append(img)
        #Viral_train.append(cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE))
        #Viral_train.append(cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
        i+=1
      elif i<(80/100)*1346:
        Viral_validate.append(img)
        i+=1
      else:
        Viral_test.append(img)
        i+=1
Viral_train=np.array(Viral_train)
Viral_validate=np.array(Viral_validate)
Viral_test=np.array(Viral_test)
print("Done")

Done


# **Combining test data , validate , train**

In [ ]:
training_data=[]
validate_data=[]
test_data=[]

for i in covid_train :
  training_data.append([i,0])
for i in Normal_train:
  training_data.append([i,1])
for i in Viral_train:
  training_data.append([i,2])

for i in covid_validate :
  validate_data.append([i,0])
for i in Normal_validate:
  validate_data.append([i,1])
for i in Viral_validate:
  validate_data.append([i,2])

for i in covid_test :
  test_data.append([i,0])
for i in Normal_test:
  test_data.append([i,1])
for i in Viral_test:
  test_data.append([i,2])

np.random.shuffle(training_data)
np.random.shuffle(validate_data)
np.random.shuffle(test_data)


#**Data Loaders**

In [ ]:
train_loader = torch.utils.data.DataLoader(training_data, batch_size=100)
validate_loader = torch.utils.data.DataLoader(validate_data, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100)

# **Fully connected neural network**




## Few Layers

In [ ]:
class FCNNFewLayers(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=256*256, out_features=1024) 
        self.fc2 = nn.Linear(in_features=1024, out_features=50)
        self.out = nn.Linear(in_features=50, out_features=3) 
    
    def forward(self, t):
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)
        t = self.out(t)
        return F.log_softmax(t)



## Many Layers

In [ ]:
class FCNNManyLayers(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=256*256, out_features=4096) 
        self.fc2 = nn.Linear(in_features=4096, out_features=1024)
        self.fc3 = nn.Linear(in_features=1024, out_features=512)
        self.fc4 = nn.Linear(in_features=512, out_features=256)
        self.fc5 = nn.Linear(in_features=256, out_features=128)
        self.fc6 = nn.Linear(in_features=128, out_features=64) 
        self.fc7 = nn.Linear(in_features=64, out_features=32)
        self.fc8 = nn.Linear(in_features=32, out_features=16)
        self.fc9 = nn.Linear(in_features=16, out_features=8)
        self.out = nn.Linear(in_features=8, out_features=3) 
    
    def forward(self, t):
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.fc3(t)
        t = F.relu(t)

        t = self.fc4(t)
        t = F.relu(t)

        t = self.fc5(t)
        t = F.relu(t)

        t = self.fc6(t)
        t = F.relu(t)

        t = self.fc7(t)
        t = F.relu(t)

        t = self.fc8(t)
        t = F.relu(t)

        t = self.fc9(t)
        t = F.relu(t)

        t = self.out(t)
        return F.log_softmax(t)
      


# **CNN**


## Few Layers

In [ ]:
class CNNFewLayers(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3,padding=1) 
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3,padding=1)

        #Output of Thepool2d layer is[100,12,64,64]
        self.fc1 = nn.Linear(in_features=12 * 64 * 64, out_features=12*3) 
        self.fc2 = nn.Linear(in_features=12*3, out_features=6*3)
        self.out = nn.Linear(in_features=6*3, out_features=3) 
    
    def forward(self, t):
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        #print("tensor shape:",t.shape)
        #Output of Thepool2d layer is[100,12,64,64]
        t = t.reshape(-1, 12 * 64 * 64)
        #(100,12*64*64)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.out(t)
        return t


## Many Layers

In [ ]:
class CNNManyLayers(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3,padding=1) 
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3,padding=1)
        
        #Output of Thepool2d layer is[100,24,32,32]
        self.fc1 = nn.Linear(in_features=24 * 32 * 32, out_features=24*3) 
        self.fc2 = nn.Linear(in_features=24*3, out_features=12*3)
        self.fc3 = nn.Linear(in_features=12*3, out_features=6*3)
        self.out = nn.Linear(in_features=6*3, out_features=3)
    
    def forward(self, t):
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv3(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        #Output of Thepool2d layer is[64,24,32,32]
        t = t.reshape(-1, 24 * 32 * 32)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.fc3(t)
        t = F.relu(t)

        t = self.out(t)
        return t



# **ResNet18**

##Pretrained


In [ ]:
resnet18Pretrained =torchvision.models.resnet18(pretrained=True)
resnet18Pretrained.fc=torch.nn.Linear(in_features=512, out_features=3)

##Untrained

In [ ]:
resnet18 =torchvision.models.resnet18(pretrained=False)
resnet18.fc=torch.nn.Linear(in_features=512, out_features=3)

# **ResNet50**

##Pretrained

In [ ]:
resnet50Pretrained =torchvision.models.resnet50(pretrained=True)
resnet50Pretrained.fc=torch.nn.Linear(in_features=2048, out_features=3)

##Untrained

In [ ]:
resnet50 =torchvision.models.resnet50(pretrained=False)
resnet50.fc=torch.nn.Linear(in_features=2048, out_features=3)

# **Trainning Loop**

In [ ]:
def Train(ModelType,optimizer):
  preds_list=[]
  label_list=[]
  total_loss = 0
  total_correct = 0
  for batch in train_loader: 
      images, labels = batch 
      #reshape from(100,256,256,3) to (100,3,256,256)
      images=images.reshape(len(images),3,256,256)
      images=images.float()
      #Fully connected
      if(ModelType.__class__.__name__=="FCNNFewLayers" or ModelType.__class__.__name__=="FCNNManyLayers"):
        images=torchvision.transforms.functional.rgb_to_grayscale(images,1)
        images = images.view(-1, 256*256) #Change shape from (100,1,256,256)->(100,256*256)
      
      preds = ModelType(images)

      for i in range(len(preds)):
        preds_list.append(preds.argmax(dim=1)[i].item())
        label_list.append(labels[i].item())

      loss = F.cross_entropy(preds, labels) # Calculate Loss
      optimizer.zero_grad()
      loss.backward() # Calculate Gradients
      optimizer.step() # Update Weights
      total_loss += loss.item()
  return total_loss,accuracy_score(label_list, preds_list),f1_score(label_list, preds_list, average=None)

# **Validation Loop**

In [ ]:
def Validate(ModelType):
  preds_list=[]
  label_list=[]
  total_loss = 0
  total_correct = 0
  for batch in validate_loader: 
      images, labels = batch 
      images=images.reshape(len(images),3,256,256)
      images=images.float()

      #Fully connected
      if(ModelType.__class__.__name__=="FCNNFewLayers" or ModelType.__class__.__name__=="FCNNManyLayers"):
        images=torchvision.transforms.functional.rgb_to_grayscale(images,1)
        images = images.view(-1, 256*256)
      
      preds = ModelType(images)

      for i in range(len(preds)):
        preds_list.append(preds.argmax(dim=1)[i].item())
        label_list.append(labels[i].item())

      loss = F.cross_entropy(preds, labels) # Calculate Loss
      total_loss += loss.item()
      total_correct =total_correct+preds.argmax(dim=1).eq(labels).sum()
  return total_loss,accuracy_score(label_list, preds_list),f1_score(label_list, preds_list, average=None)

# **Test Loop**

In [ ]:
def Test(ModelType):
  preds_list=[]
  label_list=[]
  total_loss = 0
  total_correct = 0
  for batch in test_loader: 
      images, labels = batch 
      images=images.reshape(len(images),3,256,256)
      images=images.float()

      #Fully connected
      if(ModelType.__class__.__name__=="FCNNFewLayers" or ModelType.__class__.__name__=="FCNNManyLayers"):
        images=torchvision.transforms.functional.rgb_to_grayscale(images,1)
        images = images.view(-1, 256*256)
      
      preds = ModelType(images)

      for i in range(len(preds)):
        preds_list.append(preds.argmax(dim=1)[i].item())
        label_list.append(labels[i].item())

      loss = F.cross_entropy(preds, labels) # Calculate Loss
      total_loss += loss.item()
  return total_loss,accuracy_score(label_list, preds_list),f1_score(label_list, preds_list, average=None)


# Dataloader for resnet

In [ ]:
train_loader = torch.utils.data.DataLoader(training_data, batch_size=64)
validate_loader = torch.utils.data.DataLoader(validate_data, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

# **Accurcy and f1 score report**

In [ ]:
def printResults(loss,accurcy,f1score):
  print(" loss: {:.2f} ".format(loss), end ="")
  print(" accurcy: {:.2f}% ".format(accurcy*100), end ="")
  print(" F1 of class 0: {:.2f}%  F1 of class 1: {:.2f}%  F1 of class 2: {:.2f}%".format( f1score[0]*100, f1score[1]*100, f1score[2]*100))

## Fully connected few

In [ ]:
FCNNFnetwork=FCNNFewLayers()
optimizer = optim.Adam(FCNNFnetwork.parameters(), lr=0.001)
print("Trainning Phase :")
for epoch in range(10):
  loss,accurcy,f1score=Train(FCNNFnetwork,optimizer)
  print("epoch ",epoch," : ")
  printResults(loss,accurcy,f1score)
  
print("Validation Phase :")
loss,accurcy,f1score=Validate(FCNNFnetwork)
printResults(loss,accurcy,f1score)

print("Test Phase :")
loss,accurcy,f1score=Test(FCNNFnetwork)
printResults(loss,accurcy,f1score)

Trainning Phase :


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


epoch  0  : 
 loss: 23258.37  accurcy: 38.52%  F1 of class 0: 44.34%  F1 of class 1: 34.50%  F1 of class 2: 36.67%
epoch  1  : 
 loss: 614.41  accurcy: 68.55%  F1 of class 0: 85.85%  F1 of class 1: 58.42%  F1 of class 2: 62.86%
epoch  2  : 
 loss: 352.48  accurcy: 75.55%  F1 of class 0: 91.23%  F1 of class 1: 68.39%  F1 of class 2: 68.74%
epoch  3  : 
 loss: 555.91  accurcy: 70.01%  F1 of class 0: 87.32%  F1 of class 1: 62.10%  F1 of class 2: 62.41%
epoch  4  : 
 loss: 618.05  accurcy: 71.51%  F1 of class 0: 81.82%  F1 of class 1: 66.62%  F1 of class 2: 66.91%
epoch  5  : 
 loss: 480.35  accurcy: 75.80%  F1 of class 0: 87.94%  F1 of class 1: 71.63%  F1 of class 2: 69.29%
epoch  6  : 
 loss: 315.86  accurcy: 79.11%  F1 of class 0: 91.63%  F1 of class 1: 73.72%  F1 of class 2: 73.37%
epoch  7  : 
 loss: 441.58  accurcy: 75.63%  F1 of class 0: 87.22%  F1 of class 1: 72.06%  F1 of class 2: 68.99%
epoch  8  : 
 loss: 465.12  accurcy: 76.58%  F1 of class 0: 85.81%  F1 of class 1: 72.69%  F1 

## Fully connected Many

In [ ]:
FCNNMnetwork=FCNNManyLayers()
optimizer = optim.Adam(FCNNMnetwork.parameters(), lr=0.001)
print("Trainning Phase :")
for epoch in range(10):
  loss,accurcy,f1score=Train(FCNNMnetwork,optimizer)
  print("epoch ",epoch," : ")
  printResults(loss,accurcy,f1score)
  
print("Validation Phase :")
loss,accurcy,f1score=Validate(FCNNMnetwork)
printResults(loss,accurcy,f1score)

print("Test Phase :")
loss,accurcy,f1score=Test(FCNNMnetwork)
printResults(loss,accurcy,f1score)

Trainning Phase :


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch  0  : 
 loss: 144.38  accurcy: 33.42%  F1 of class 0: 32.59%  F1 of class 1: 27.73%  F1 of class 2: 38.15%
epoch  1  : 
 loss: 27.46  accurcy: 35.86%  F1 of class 0: 37.81%  F1 of class 1: 38.62%  F1 of class 2: 30.23%
epoch  2  : 
 loss: 19.64  accurcy: 55.98%  F1 of class 0: 77.14%  F1 of class 1: 49.03%  F1 of class 2: 38.36%
epoch  3  : 
 loss: 15.91  accurcy: 62.63%  F1 of class 0: 90.38%  F1 of class 1: 50.48%  F1 of class 2: 49.60%
epoch  4  : 
 loss: 15.42  accurcy: 65.08%  F1 of class 0: 90.64%  F1 of class 1: 54.34%  F1 of class 2: 52.93%
epoch  5  : 
 loss: 14.83  accurcy: 66.92%  F1 of class 0: 90.62%  F1 of class 1: 59.35%  F1 of class 2: 52.96%
epoch  6  : 
 loss: 14.32  accurcy: 69.67%  F1 of class 0: 89.96%  F1 of class 1: 64.83%  F1 of class 2: 55.75%
epoch  7  : 
 loss: 13.65  accurcy: 72.84%  F1 of class 0: 90.79%  F1 of class 1: 67.05%  F1 of class 2: 62.77%
epoch  8  : 
 loss: 12.24  accurcy: 77.26%  F1 of class 0: 91.54%  F1 of class 1: 72.96%  F1 of class 2

## CNN Few

In [ ]:
CNNFnetwork=CNNFewLayers()
optimizer = optim.Adamax(CNNFnetwork.parameters(), lr=0.01)
print("Trainning Phase :")
for epoch in range(10):
  loss,accurcy,f1score=Train(CNNFnetwork,optimizer)
  print("epoch",epoch,": ")
  printResults(loss,accurcy,f1score)
  
print("Validation Phase :")
loss,accurcy,f1score=Validate(CNNFnetwork)
printResults(loss,accurcy,f1score)

print("Test Phase :")
loss,accurcy,f1score=Test(CNNFnetwork)
printResults(loss,accurcy,f1score)

Trainning Phase :
epoch 0 : 
 loss: 4654.41  accurcy: 50.19%  F1 of class 0: 63.31%  F1 of class 1: 56.47%  F1 of class 2: 14.54%
epoch 1 : 
 loss: 17.68  accurcy: 64.52%  F1 of class 0: 86.58%  F1 of class 1: 63.98%  F1 of class 2: 37.27%
epoch 2 : 
 loss: 10.99  accurcy: 80.05%  F1 of class 0: 92.18%  F1 of class 1: 75.27%  F1 of class 2: 74.11%
epoch 3 : 
 loss: 9.04  accurcy: 85.59%  F1 of class 0: 93.66%  F1 of class 1: 82.96%  F1 of class 2: 81.13%
epoch 4 : 
 loss: 8.87  accurcy: 86.23%  F1 of class 0: 93.08%  F1 of class 1: 84.37%  F1 of class 2: 82.11%
epoch 5 : 
 loss: 8.38  accurcy: 86.23%  F1 of class 0: 94.31%  F1 of class 1: 83.46%  F1 of class 2: 81.91%
epoch 6 : 
 loss: 7.58  accurcy: 87.95%  F1 of class 0: 94.50%  F1 of class 1: 85.86%  F1 of class 2: 84.19%
epoch 7 : 
 loss: 6.73  accurcy: 88.55%  F1 of class 0: 95.40%  F1 of class 1: 86.45%  F1 of class 2: 84.54%
epoch 8 : 
 loss: 6.74  accurcy: 88.67%  F1 of class 0: 95.13%  F1 of class 1: 86.97%  F1 of class 2: 84.

## CNN Many

In [ ]:
CNNMnetwork=CNNManyLayers()
optimizer = optim.Adamax(CNNMnetwork.parameters(), lr=0.01)
print("Trainning Phase :")
for epoch in range(10):
  loss,accurcy,f1score=Train(CNNMnetwork,optimizer)
  print("epoch ",epoch," : ")
  printResults(loss,accurcy,f1score)
  
print("Validation Phase :")
loss,accurcy,f1score=Validate(CNNMnetwork)
printResults(loss,accurcy,f1score)

print("Test Phase :")
loss,accurcy,f1score=Test(CNNMnetwork)
printResults(loss,accurcy,f1score)

Trainning Phase :
epoch  0  : 
 loss: 358.45  accurcy: 44.23%  F1 of class 0: 54.13%  F1 of class 1: 42.99%  F1 of class 2: 32.11%
epoch  1  : 
 loss: 13.19  accurcy: 74.43%  F1 of class 0: 88.32%  F1 of class 1: 73.03%  F1 of class 2: 62.26%
epoch  2  : 
 loss: 8.94  accurcy: 84.81%  F1 of class 0: 91.96%  F1 of class 1: 83.20%  F1 of class 2: 79.95%
epoch  3  : 
 loss: 7.30  accurcy: 87.69%  F1 of class 0: 92.12%  F1 of class 1: 87.21%  F1 of class 2: 84.15%
epoch  4  : 
 loss: 6.01  accurcy: 90.00%  F1 of class 0: 92.91%  F1 of class 1: 90.53%  F1 of class 2: 86.83%
epoch  5  : 
 loss: 4.88  accurcy: 92.36%  F1 of class 0: 94.13%  F1 of class 1: 93.34%  F1 of class 2: 89.75%
epoch  6  : 
 loss: 4.24  accurcy: 93.22%  F1 of class 0: 95.10%  F1 of class 1: 93.72%  F1 of class 2: 91.00%
epoch  7  : 
 loss: 3.57  accurcy: 94.17%  F1 of class 0: 96.11%  F1 of class 1: 93.99%  F1 of class 2: 92.59%
epoch  8  : 
 loss: 3.47  accurcy: 94.59%  F1 of class 0: 96.89%  F1 of class 1: 94.22%  F1

## Resnet-18 Pretrained

In [ ]:
optimizer=optim.Adamax(resnet18Pretrained.parameters(), lr=0.01)

resnet18Pretrained.train()
print("Trainning Phase :")
for epoch in range(5):
  loss,accurcy,f1score=Train(resnet18Pretrained,optimizer)
  print("epoch ",epoch," : ")
  printResults(loss,accurcy,f1score)

resnet18Pretrained.eval()
print("Validation Phase :")
loss,accurcy,f1score=Validate(resnet18Pretrained)
printResults(loss,accurcy,f1score)

print("Test Phase :")
loss,accurcy,f1score=Test(resnet18Pretrained)
printResults(loss,accurcy,f1score)

## Resnet-18 Untrained

In [ ]:
optimizer=optim.Adamax(resnet18.parameters(), lr=0.01)

resnet18.train()
print("Trainning Phase :")
for epoch in range(5):
  loss,accurcy,f1score=Train(resnet18,optimizer)
  print("epoch ",epoch," : ")
  printResults(loss,accurcy,f1score)

resnet18.eval()
print("Validation Phase :")
loss,accurcy,f1score=Validate(resnet18)
printResults(loss,accurcy,f1score)

print("Test Phase :")
loss,accurcy,f1score=Test(resnet18)
printResults(loss,accurcy,f1score)

Trainning Phase :
epoch  0  : 
 loss: 36.99  accurcy: 62.85%  F1 of class 0: 76.42%  F1 of class 1: 61.94%  F1 of class 2: 50.74%
epoch  1  : 
 loss: 18.95  accurcy: 79.24%  F1 of class 0: 89.52%  F1 of class 1: 78.10%  F1 of class 2: 70.01%
epoch  2  : 
 loss: 15.57  accurcy: 84.21%  F1 of class 0: 91.44%  F1 of class 1: 82.43%  F1 of class 2: 79.47%
epoch  3  : 
 loss: 12.13  accurcy: 88.50%  F1 of class 0: 92.88%  F1 of class 1: 87.26%  F1 of class 2: 85.82%
epoch  4  : 
 loss: 10.94  accurcy: 90.09%  F1 of class 0: 93.25%  F1 of class 1: 89.89%  F1 of class 2: 87.49%
Validation Phase :
 loss: 4.67  accurcy: 87.00%  F1 of class 0: 90.55%  F1 of class 1: 86.86%  F1 of class 2: 83.69%
Test Phase :
 loss: 4.87  accurcy: 85.73%  F1 of class 0: 90.87%  F1 of class 1: 86.56%  F1 of class 2: 79.76%


## Resnet-50 Pretrained

In [ ]:
optimizer=optim.Adamax(resnet50Pretrained.parameters(), lr=0.01)
resnet50Pretrained.train()
print("Trainning Phase :")
for epoch in range(5):
  loss,accurcy,f1score=Train(resnet50Pretrained,optimizer)
  print("epoch ",epoch," : ")
  printResults(loss,accurcy,f1score)

resnet50Pretrained.eval()
print("Validation Phase :")
loss,accurcy,f1score=Validate(resnet50Pretrained)
printResults(loss,accurcy,f1score)

print("Test Phase :")
loss,accurcy,f1score=Test(resnet50Pretrained)
printResults(loss,accurcy,f1score)

Trainning Phase :
epoch  0  : 
 loss: 35.25  accurcy: 57.06%  F1 of class 0: 76.21%  F1 of class 1: 54.54%  F1 of class 2: 43.34%
epoch  1  : 
 loss: 23.62  accurcy: 73.36%  F1 of class 0: 88.90%  F1 of class 1: 67.28%  F1 of class 2: 65.24%
epoch  2  : 
 loss: 16.08  accurcy: 83.83%  F1 of class 0: 92.04%  F1 of class 1: 80.93%  F1 of class 2: 79.30%


## Resnet-50 Untrained

In [ ]:
optimizer=optim.Adamax(resnet50.parameters(), lr=0.01)
resnet50.train()
print("Trainning Phase :")
for epoch in range(5):
  loss,accurcy,f1score=Train(resnet50,optimizer)
  print("epoch ",epoch," : ")
  printResults(loss,accurcy,f1score)

resnet50.eval()
print("Validation Phase :")
loss,accurcy,f1score=Validate(resnet50)
printResults(loss,accurcy,f1score)

print("Test Phase :")
loss,accurcy,f1score=Test(resnet50)
printResults(loss,accurcy,f1score)

Trainning Phase :
epoch  0  : 
 loss: 48.12  accurcy: 59.93%  F1 of class 0: 79.94%  F1 of class 1: 47.22%  F1 of class 2: 54.01%
epoch  1  : 
 loss: 22.40  accurcy: 73.57%  F1 of class 0: 91.36%  F1 of class 1: 66.75%  F1 of class 2: 64.06%
epoch  2  : 
 loss: 16.79  accurcy: 81.81%  F1 of class 0: 91.77%  F1 of class 1: 78.11%  F1 of class 2: 76.40%
epoch  3  : 
 loss: 13.80  accurcy: 86.14%  F1 of class 0: 93.17%  F1 of class 1: 83.32%  F1 of class 2: 82.61%
epoch  4  : 
 loss: 11.89  accurcy: 88.46%  F1 of class 0: 94.18%  F1 of class 1: 86.08%  F1 of class 2: 85.61%
Validation Phase :
